In [2]:
import pandas as pd
df = pd.DataFrame( {'a':['C1','C1','C2','C2','C2','C3'], 't':[1,1,2,1,3,1],'b':['A','B','C','C','D','E']})   #ID、時間、行為

In [3]:
df    # a : 顧客 (ID)  t : 時間   , b:行為

,a,t,b
0,C1,1,A
1,C1,1,B
2,C2,2,C
3,C2,1,C
4,C2,3,D
5,C3,1,E


In [4]:
#想了解每個人的購物行為，根據ID和時間做排序
df1 = df.sort_values(['a','t'],ascending=True)      #先用ID排完換用時間排序
df1

,a,t,b
0,C1,1,A
1,C1,1,B
3,C2,1,C
2,C2,2,C
4,C2,3,D
5,C3,1,E


# 連續重複刪除  

如C2 在兩個不同時間點t=2,t=3時，做了相同的行為C，要把它視為一筆就好

In [5]:
df1 = df1[(df1['b'].shift() != df1['b']) | (df1['a'].shift() != df1['a'])]   #shift():上下挪移      若行為不同或時間不同，則保留該列
df1                                             #目的: 刪除重複的b欄位資料
                                                 #做法: 將b欄位上下一格看是否不等於'b' 或 將a欄位上下一格看是否不等於'a' 給抓出來
                                                # (只要不同使用者但相同時間 或 相同使用者但不同時間) 的資料都要抓出來

,a,t,b
0,C1,1,A
1,C1,1,B
3,C2,1,C
4,C2,3,D
5,C3,1,E


將相同的人a  在同個時間點t 的行為b 做合併

In [6]:
df2 = df1.groupby(['a','t'])['b'].apply(''.join).reset_index()  #想像成key,Value   若key對應到的欄位相同，則整合成一個就好，
                                              #多個key對應到多個欄位也是整合成一個。      而.apply()是針對每個分組進行相對應的數據處理
                                            #若在相同的a和相同的下，將b 加在一起   ，表示同個時間點下發生的所有行為
df2 = pd.DataFrame(df2)
df2

,a,t,b
0,C1,1,AB
1,C2,1,C
2,C2,3,D
3,C3,1,E


In [7]:
df2["b"] = "(" + df2["b"] +")"
df2

,a,t,b
0,C1,1,(AB)
1,C2,1,(C)
2,C2,3,(D)
3,C3,1,(E)


# 資料整理 ->  b欄位的行為已經被依時間區分出

# 最終目的要把結果寫成一列表示

In [8]:
df3 = df2.groupby(['a'])['b'].apply(''.join).reset_index(name='new')  #將相同的整合成一個，前面已經用時間區分出不同的行為了
                                                                    #利用.apply()將他們加在同一列 
df3 = pd.DataFrame(df3)
df3

,a,new
0,C1,(AB)
1,C2,(C)(D)
2,C3,(E)


In [9]:
out = ' '.join(df3["new"])         #用空格區分不同的消費者
out

'(AB) (C)(D) (E)'

也可將相同的a欄位做合併，並將b欄位用apply()合併成list


In [32]:
df7 = df1
df7

,a,t,b
0,C1,1,A
1,C1,1,B
3,C2,1,C
4,C2,3,D
5,C3,1,E


In [34]:
df7 = df7.groupby(['a','t'])['b'].apply(''.join).reset_index()
df7

,a,t,b
0,C1,1,AB
1,C2,1,C
2,C2,3,D
3,C3,1,E


In [36]:
df_list = df7.groupby(['a'])['b'].apply(list).reset_index()
df_list

,a,b
0,C1,[AB]
1,C2,"[C, D]"
2,C3,[E]


In [37]:
df_list[['b']]

,b
0,[AB]
1,"[C, D]"
2,[E]


In [38]:
output = ' '.join(df_list['b'].astype(str))
output

"['AB'] ['C', 'D'] ['E']"